In [112]:
import numpy as np
from queue import PriorityQueue
from collections import defaultdict
import copy
import heapq

In [113]:
def read_data(fname):
    data = []
    with open(fname, 'r') as inf:
        for line in inf.readlines():
            if line.strip() == '':
                continue
            data.append([c for c in line.strip()])

    return np.array(data)

In [114]:
def longest_path(data):
    # vismap = np.zeros(data.shape, dtype=int)
    # vismap[np.where(data == '#')] = 1
    # visited = {0: vismap}
    visited = defaultdict(set)
    # lmap = np.ones(data.shape, dtype=int)
    # lmap *= -1
    # pathlen = {0: lmap}
    # newpathlen = {}
    nextpath = 1
    # newpaths = set()

    ty = data.shape[0]-1
    tx = np.where(data[ty, :]=='.')[0][0]

    q = []

    x = np.where(data[0, :]=='.')[0][0]
    heapq.heappush(q, (0, (0, 0, x)))

    maxlen = 0

    while len(q):
        p, (l, y, x) = heapq.heappop(q)
        visited[p].add((y, x))
        if (y, x) == (ty, tx):
            maxlen = max(maxlen, l)
            del visited[p]
            continue

        newpath = False
        for dy, dx in ((0, 1), (-1, 0), (0, -1), (1, 0)):
            ny, nx = y+dy, x+dx
            if ny < 0 or ny >= data.shape[0] or x < 0 or x >= data.shape[1]:
                continue
            if (ny, nx) in visited[p]:
                continue

            if data[ny, nx] == '.':
                if newpath:
                    visited[nextpath] = copy.copy(visited[p])
                    heapq.heappush(q, (nextpath, (l+1, ny, nx)))
                    nextpath += 1
                else:
                    heapq.heappush(q, (p, (l+1, ny, nx)))
                newpath = True
            elif data[ny, nx] == '>' and (dy, dx) == (0, 1):
                if newpath:
                    visited[nextpath] = copy.copy(visited[p])
                    heapq.heappush(q, (nextpath, (l+2, ny+dy, nx+dx)))
                    nextpath += 1
                    newpath = True
                else:
                    heapq.heappush(q, (p, (l+2, ny+dy, nx+dx)))
                    newpath = True
                newpath = True
            elif data[ny, nx] == '<' and (dy, dx) == (0, -1):
                if newpath:
                    visited[nextpath] = copy.copy(visited[p])
                    heapq.heappush(q, (nextpath, (l+2, ny+dy, nx+dx)))
                    nextpath += 1
                    newpath = True
                else:
                    heapq.heappush(q, (p, (l+2, ny+dy, nx+dx)))
                    newpath = True
                newpath = True
            elif data[ny, nx] == 'v' and (dy, dx) == (1, 0):
                if newpath:
                    visited[nextpath] = copy.copy(visited[p])
                    heapq.heappush(q, (nextpath, (l+2, ny+dy, nx+dx)))
                    nextpath += 1
                    newpath = True
                else:
                    heapq.heappush(q, (p, (l+2, ny+dy, nx+dx)))
                    newpath = True
                newpath = True
            elif data[ny, nx] == '^' and (dy, dx) == (-1, 0):
                if newpath:
                    visited[nextpath] = copy.copy(visited[p])
                    heapq.heappush(q, (nextpath, (l+2, ny+dy, nx+dx)))
                    nextpath += 1
                    newpath = True
                else:
                    heapq.heappush(q, (p, (l+2, ny+dy, nx+dx)))
                    newpath = True
                newpath = True
            else:
                continue

    return maxlen

                    
        
                

    

In [115]:
def follow_path(node, finalnode, path, pathlen, plen, paths, fullpaths):
    path.append(node)

    for n in paths[node]:
        if n not in path:
            if n == finalnode:
                path.append(n)
                fullpaths.append((path, plen+pathlen[node][n]))
                return

            nextpath = copy.copy(path)
            follow_path(n, finalnode, nextpath, pathlen, plen+pathlen[node][n], paths, fullpaths)
            
def find_nodes(data):
    pathmap = np.zeros(data.shape, dtype=int)
    nodes = {}
    rnodes = {}
    node_counter = 1
    for y in range(0, data.shape[0]):
        for x in range(0, data.shape[1]):
            if data[y, x] == '#':
                continue
            npaths = 0
            for dy, dx in ((0, 1), (-1, 0), (0, -1), (1, 0)):
                ny = y+dy
                nx = x+dx
                if ny < 0 or ny>=data.shape[0]:
                    continue
                if nx < 0 or nx>=data.shape[1]:
                    continue
                if data[ny, nx] != '#':
                    npaths += 1
            pathmap[y, x] = npaths
            if npaths == 1 or npaths > 2:
                nodes[node_counter] = (y,x)
                rnodes[(y, x)] = node_counter
                node_counter += 1

    start_node = 1
    final_node = node_counter-1
    # print(f'Found {len(nodes)} nodes')

    paths = {}
    pathlen = defaultdict(dict)

    for p, (y_s, x_s) in nodes.items():
        paths[p] = set()
        for dy_1, dx_1 in ((0, 1), (1, 0), (0, -1), (-1, 0)):
            # Test all paths leading from this node
            y = y_s+dy_1
            x = x_s+dx_1
            l = 2
            if y < 0 or y>=data.shape[0]:
                continue
            if x < 0 or x>=data.shape[1]:
                continue
            if pathmap[y, x] == 0:
                continue
            done = False
            visited = {(y_s, x_s)}
            while not done:
                visited.add((y, x))
                done = True
                for dy, dx in ((0, 1), (-1, 0), (0, -1), (1, 0)):
                    ny = y+dy
                    nx = x+dx
                    if (ny, nx) in visited:
                        continue
                    if ny < 0 or ny>=data.shape[0]:
                        continue
                    if nx < 0 or nx>=data.shape[1]:
                        continue
                    if pathmap[ny, nx] == 0:
                        continue
                    if pathmap[ny, nx] != 2:
                        if rnodes[(ny, nx)] == p:
                            print('Found a loop!')
                            break
                        # Found a node
                        paths[p].add(rnodes[(ny, nx)])
                        pathlen[p][rnodes[(ny, nx)]] = l
                        pathlen[rnodes[(ny, nx)]][p] = l
                        done = True
                        break
                    else:
                        l += 1
                        y = ny
                        x = nx
                        done = False
                        break

    fp = []
    follow_path(1, final_node, [], pathlen, 0, paths, fp)  # node, finalnode, path, pathlen, plen, paths, fullpaths):

    maxlen = 0
    for p, pl in fp:
        maxlen=max(maxlen, pl)

    return maxlen

In [117]:
print('*****\nPuzzle1\n*****\n')

print('Test case\n')

plen = longest_path(read_data('input23a.txt'))

print(f'Longest path is {plen}')

assert plen == 94

print('\nPuzzle case\n')

plen = longest_path(read_data('input23.txt'))

print(f'Longest path is {plen}')

assert plen == 2310

print('\n*****\nPuzzle2\n*****\n')

plen = find_nodes(read_data('input23a.txt'))

print(f'Longest path is now {plen}')

assert plen == 154

print('\nPuzzle case\n')

plen = find_nodes(read_data('input23.txt'))

print(f'Longest path is now {plen}')

assert plen == 6738

*****
Puzzle1
*****

Test case

Longest path is 94

Puzzle case

Longest path is 2310

*****
Puzzle2
*****

Longest path is now 154

Puzzle case

Longest path is now 6738
